## [Bayesian Factorization Machines with Stan and R](https://nyhackr.blob.core.windows.net/presentations/A_Common_Model_Separated_by_Two_Disciplines-Adam_Lauretig.pdf)

### [Mastering Shiny](https://mastering-shiny.org/basic-intro.html)

In [1]:
library(MASS)
library(Matrix)
library(trialr)

Loading required package: Rcpp

Warning message in paste0("stan_fit4", names(stanmodels), "_mod"):
"strings not representable in native encoding will be translated to UTF-8"
Warning message in (function (n) :
"strings not representable in native encoding will be translated to UTF-8"
Warning message in (function (n) :
"strings not representable in native encoding will be translated to UTF-8"
Warning message in (function (n) :
"strings not representable in native encoding will be translated to UTF-8"
Warning message in (function (n) :
"strings not representable in native encoding will be translated to UTF-8"
Warning message in (function (n) :
"strings not representable in native encoding will be translated to UTF-8"


In [2]:
library(rstan)
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.8, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



### Generating Simulation Data

In [3]:
set.seed(111)

In [4]:
N <- 50
group_1 <- paste0('i', 1:N)

In [5]:
J <- 10
group_2 <- paste0('j', 1:J)

In [6]:
K <- 5

In [7]:
predictors <- expand.grid(group_1=group_1, group_2=group_2)

In [8]:
X_mat <- sparse.model.matrix(~ factor(group_1) + factor(group_2) -1, data=predictors)

In [9]:
predictors_as_numeric <- cbind(as.numeric(factor(predictors[,1])), as.numeric(factor(predictors[,2])))

In [10]:
betas <- matrix(rnorm(n=ncol(X_mat), 0,2))

In [11]:
linear_predictors = X_mat %*% betas  # y

### [LKJ](https://yingqijing.medium.com/lkj-correlation-distribution-in-stan-29927b69e9be)

#### if eta > 1, the correlation values in correlation matrices are going to centered around 0. higher eta indicate no correlations (converge to identity correlation matrix).

![](https://miro.medium.com/max/1400/1*v2LSgSMjYwCjWYpXtk437g.webp)

In [12]:
gamma_omega <- rlkjcorr(n = 1, K = K, eta = 0.9)#gamma_omega_prior)

In [13]:
delta_omega <- rlkjcorr(n=1, K=K, eta=0.5)

In [14]:
gammas <- mvrnorm(n=N, mu=rep(0,K), Sigma = gamma_omega)

In [15]:
deltas <- mvrnorm(n=J, mu=rep(0,K), Sigma = delta_omega)

In [16]:
factor_terms = matrix(NA, nrow=nrow(linear_predictors), ncol=1)

In [17]:
dim(factor_terms)

[1] 500   1

In [18]:
for (i in 1:nrow(predictors)){
    g1 <- as.character(predictors[i,1])
    g1 <- as.numeric(substring(g1, 2, nchar(g1)))

    g2 <- as.character(predictors[i,2])
    g2 <- as.numeric(substring(g2, 2, nchar(g2)))

    #print(paste0(g1, g2))
    factor_terms[i, ] =  matrix(gammas[g1,], nrow=1) %*% matrix(deltas[g2,], ncol=1)   
}

In [19]:
y <- linear_predictors + factor_terms + rnorm(n=nrow(linear_predictors), 0, 0.1)

In [20]:
data_list = list(
    N = N,
    J = J,
    K = K,
    X = predictors_as_numeric,
    y = y[,1],
    beta_sigma = 1,
    y_sigma = 0.1
)

In [21]:
saveRDS(data_list, file = 'data_list.rds')

In [23]:
model = stan_model('../RStan/Models/factorization.stan')

In [24]:
remove(fit)
fit <- sampling(object = model,
                data = data_list,
                init = "random",
                control = list(adapt_delta = 0.95),
                chains = 4,
                iter = 10,
                warmup = 5,
                thin = 1,
                verbose = TRUE)

Warning message in remove(fit):
"object 'fit' not found"



CHECKING DATA AND PREPROCESSING FOR MODEL 'factorization' NOW.

COMPILING MODEL 'factorization' NOW.

STARTING SAMPLER FOR MODEL 'factorization' NOW.


Warning message:
"There were 20 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is Inf, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat"
